In [50]:
import pandas as pd 
import numpy as np
import pyblp
from sklearn.linear_model import LinearRegression

In [51]:
product_data = pd.read_csv('../data/market_integrates_1.csv')
print(product_data)

     market_ids  firm_ids  characteristic1  characteristic2    prices  \
0             1         1         3.222649         1.217506  5.057337   
1             1         2         3.263651         1.515093  4.808937   
2             1         3         4.932264         2.045617  4.516279   
3             1         4         3.516152         3.611320  4.596241   
4             1         5         1.570786         5.165628  4.580161   
..          ...       ...              ...              ...       ...   
995         100         6         1.069992         1.840119  5.472291   
996         100         7         5.183309         2.230919  4.578464   
997         100         8         5.211012         4.522679  4.095501   
998         100         9         4.434760         2.509379  4.555555   
999         100        10         1.250220         4.450655  4.737362   

     marginal_cost    shares   profits   markups  e_quantity    capital  \
0         4.261728  0.001142  0.000909  1.186687

In [52]:
# Getting the dataset estimation ready with at least a supply_side instrument/ demand_side instrument 
product_data_complete = product_data.rename(columns={'labor':'supply_instrument0',
                                                      'capital':'supply_instruments1',
                                                        # 'marginal_cost': 'supply_instrument2'
                                                        })



In [53]:
# the endog regressor
X1_formulation = pyblp.Formulation('1  + prices+ characteristic1 + characteristic2')

# the only regressor on which we want random coeff 
X2_formulation = pyblp.Formulation('0 +  prices')

# supply side charact 
# X3_formulation = pyblp.Formulation('1 + log(labor) + log(capital)')

# formulation for the cost thing (assume cost fomrulation being labor and capital)
product_formulations = (X1_formulation,
                         X2_formulation
                           #  X3_formulation
                           )
print(product_formulations)

(1 + prices + characteristic1 + characteristic2, prices)


In [54]:
mc_integration = pyblp.Integration('monte_carlo', size=1000, specification_options={'seed': 0})


In [55]:
mc_problem = pyblp.Problem(product_formulations, product_data_complete
                           , integration=mc_integration, add_exogenous=True)
print(mc_problem)

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     F     I      K1    K2    MD 
---  ----  ---  ------  ----  ----  ----
100  1000  10   100000   4     1     3  

Formulations:
       Column Indices:           0       1            2                3       
-----------------------------  ------  ------  ---------------  ---------------
 X1: Linear Characteristics      1     prices  characteristic1  characteristic2
X2: Nonlinear Characteristics  prices                                          
Dimensions:
 T    N     F     I      K1    K2    MD 
---  ----  ---  ------  ----  ----  ----
100  1000  10   100000   4     1     3  

Formulations:
       Column Indices:           0       1            2                3       
-----------------------------  ------  ------  ---------------  ---------------
 X1: Linear Characteristics      1     prices  characteristic1  characteristic2
X2: Nonlinear Characteristics  prices                                 

In [56]:
bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-4})


In [57]:
results1 = mc_problem.solve(sigma=np.ones((1, 1)), optimization=bfgs)


Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:     prices    
------  -------------
prices  +1.000000E+00
Starting optimization ...



GMM   Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Gradient                  
Step   Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement       Norm           Theta    
----  ------------  -----------  -----------  -----------  -------  -------------  -------------  -------------  -------------
 1         0             1           568         1799         0     +1.173851E-24                 +5.418664E-10  +1.000000E+00

Optimization completed after 00:00:01.
Computing the Hessian and and updating the weighting matrix ...
Computed results after 00:00:02.

Problem Results Summary:
GMM     Objective      Gradient                    Clipped  Weighting Matrix
Step      Value          Norm          Hessian     Shares   Condition Number
----  -------------  -------------  -------------  -------  ----------------
 1    +1.165448E-24  +5.401608E-10  -2.824702E-02     0      +3.094044E+02  

Starting optimizatio